In [ ]:
from nbtemplate import display_header, get_path
display_header('Subaperturing.ipynb')

# Spectral resolution and subaperturing

In these notes, I'm looking at the effect that subaperturing has on the spectral resolving power for Lynx. Since most parts of the Lynx design are still in the air, there are a lot of assumptions in here, some of which might have a major influence on the result. This is a document in progress that must evolve with the evolving mission design.

Please contact me (Moritz) if you want to use any of the figures or plots below for presentations or further reseach and I will prepare the high-resolution figures for you or provide you with the datasets behind the plot in numeric form. Please do not copy and paste the figures here; they are intentionally made at a lower resolution to optimize the display on a webpage.

In [ ]:
import sys
import numpy as np
from astropy.coordinates import SkyCoord
from astropy.table import Table, join
import astropy.units as u
import marxs
from marxs import visualization

from marxs.source import PointSource, FixedPointing, JitterPointing
from marxs.analysis import resolvingpower_from_photonlist
from marxs.simulator import Sequence

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
from marxslynx import lynx
from marxslynx.ralfgrating import facet_table, order_selector_Si, order_selector_SiPt
import marxslynx
from marxslynx.simulations import run_monoenergetic_simulation

In [ ]:
from cycler import cycler
# color is modified from standard matplotlib to match the length of the second term (12 elements)
custom_cycler = (cycler(color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf', 'r', 'b']) +
                  cycler(linewidth=[2,1,3]) * cycler(linestyle=['-', '--', ':', '-.']))

## Zeroth order image

First, we look at a zeroth order image. That allows us to check that we got the mirror right. This simulation uses a simplified mirror model. It does not simulate individual mirror shells, but instead is treats the mirror as a continuum. Every photon hitting the mirror plane is redicted to the focal point assuming an "ideal" mirror. Then, additional scatter is added both in the plane of reflection and out of the plane of reflection. Typically, for an individual shell, the scatter is larger in the plane of reflection due to figure errors and the scattering by particulates.

In this case, we want to simulate a mirror where the PSF is domianted by the mis-pointing of individual mirror shells (which is the case where sub-aperturing is the least useful). Mis-pointing of the shells happens in both x and y direction, which, looking at a single photon, can be simulated by assuming a perfect mirror and scatter of the same magnitude in the plane of the scattering and out of the plane of the scattering. 
Note that for simulations of gratings it would be cheating to apply a blur in the focal plane to account for the PSF in the way that many simpler simulations programs do it, because we need to know the direction of the photon already when it hits the gratings, and not just in the focal plane.

In [ ]:
o7triplet = (21.8 * u.Angstrom).to(u.keV, equivalencies=u.spectral())
single_en = 0.6 * u.keV

In [ ]:
# Do this in different cells intitally, so I don't have to rerun everything while debugging the code
instrum = {'2050': lynx.PerfectLynx(conf=lynx.conf)}

In [ ]:
instrum['2050_align'] = lynx.Lynx(conf=lynx.conf)

In [ ]:
instrum['5050'] = lynx.PerfectLynx(conf=lynx.conf_5050)

In [ ]:
instrum['chirp'] = lynx.PerfectLynx(conf=lynx.conf_chirp)

In [ ]:
import copy
conf_scat = copy.copy(lynx.conf)
conf_scat['inplanescatter'] = 3.5e-6 * u.rad
conf_scat['perpplanescatter'] = .7e-6 * u.rad
instrum['scat'] = lynx.PerfectLynx(conf=conf_scat)

In [ ]:
from marxslynx.bendgratings import bend_gratings
from marxs.missions.mitsnl.catgrating import CATL1L2Stack

bend = lynx.PerfectLynx(conf=lynx.conf)
bend_gratings(conf_scat, bend.elements_of_class(CATL1L2Stack), r=9500)
instrum['bend'] = bend

In [ ]:
facettab = {}
for k in instrum:
    facettab[k] = facet_table(instrum[k].elements[2])

In [ ]:
np.seterr(invalid='raise')

In [ ]:
phot = {}
for k in instrum:
    phot[k] = join(run_monoenergetic_simulation(instrum[k], single_en), facettab[k], join_type='left')

In [ ]:
photlong = {}
for k in instrum:
    photlong[k] = join(run_monoenergetic_simulation(instrum[k], single_en, n_photons=2e5 * u.s), facettab[k], join_type='left')

In [ ]:
np.clip?

In [ ]:
labels = {'2050': '50*20 mm (perfect)',
          '2050_align': '50*20 mm (alignment)',
          'scat': '50*20 mm (scatter)',
          '5050': '50*50 mm',
          'chirp': '80*160 mm (chirped)',
          
          'bend': '50*20 mm (bend)'
         }
labels2line = {'2050': '50*20 mm',
               '2050_align': '50*20 mm\n(incl. alignment)',
               'scat': '50*20 mm\nscatter',
               '5050': '50*50 mm',
               'chirp': '80*160 mm\nchirped',
               'bend': '50*20 mm\nbend'    
            }

In [ ]:
for k, v in facettab.items():
    print('{} has {} facets'.format(k, len(v)))

In [ ]:
#out = plt.hist(np.rad2deg(phot['bend']['blaze']), bins=np.arange(1.3, 1.9, .01), label=labels['bend'])
for k in labels:
    #if k == 'bend':
    #    continue
    out = plt.hist(np.rad2deg(phot[k]['blaze']), bins=np.arange(1.3, 1.9, .01), label=labels[k], lw=2, histtype='step')
plt.xlabel('Blaze angle [deg]')
plt.ylabel('Number of photons')
plt.legend()
plt.savefig(get_path('figures') + '/blaze.png', dpi=300)
plt.savefig(get_path('figures') + '/blaze.pdf', bbox_inches='tight')

In [ ]:
from matplotlib.ticker import NullFormatter

photons = phot['chirp']
ind = (photons['order'] == 0) & (photons['probability'] > 0) & (np.isfinite(photons['microcal_x']))
x = photons[ind]['microcal_x']
y = photons[ind]['microcal_y']

nullfmt = NullFormatter()         # no labels
# definitions for the axes
left, width = 0.1, 0.65
bottom, height = 0.1, 0.65
bottom_h = left_h = left + width + 0.02

rect_scatter = [left, bottom, width, height]
rect_histx = [left, bottom_h, width, 0.2]
rect_histy = [left_h, bottom, 0.2, height]

# start with a rectangular Figure
plt.figure(1, figsize=(5, 5))

axScatter = plt.axes(rect_scatter)
axHistx = plt.axes(rect_histx)
axHisty = plt.axes(rect_histy)

# no labels
axHistx.xaxis.set_major_formatter(nullfmt)
axHisty.yaxis.set_major_formatter(nullfmt)

# the scatter plot:
axScatter.scatter(x, y)
binwidth = 0.0025
xymax = np.max([np.max(np.fabs(x)), np.max(np.fabs(y))])
lim = (int(xymax/binwidth) + 1) * binwidth

axScatter.set_xlim((-lim, lim))
axScatter.set_ylim((-lim, lim))

bins = np.arange(-lim, lim + binwidth, binwidth)

len(bins)
histx = axHistx.hist(x, bins=bins)
histy = axHisty.hist(y, bins=bins, orientation='horizontal')

axHistx.set_xlim(axScatter.get_xlim())
axHisty.set_ylim(axScatter.get_ylim())

axScatter.set_xlabel('dispersion direction [mm]')
axScatter.set_ylabel('crossdispersion direction [mm]')

In [ ]:
def hpd(x, y):
    '''Simple estimate for the half-power-diameter
    '''
    r = np.sqrt((x - x.mean())**2 + (y - y.mean())**2)
    return np.median(r)

In [ ]:
print('0.5 arcsec correspond to {:6.3f} mm.'.format(np.deg2rad(0.5/3600.) * 10000.))
for k in phot:
    ind0 = (phot[k]['order'] == 0) & np.isfinite(phot[k]['microcal_y'])
    print(k, '-- Estimate for HPD [in mm]: {:6.3f}'.format(hpd(phot[k]['microcal_x'][ind0], phot[k]['microcal_y'][ind0])))

These numbers indicates that we set up our mirror correctly such that the HPD is close to 0.5 arcsec. Looking at the histogram in the plot, the shape of the PSF is also roughly Gaussian, so that these simulations can be compared with other efforts for the Lynx development, where similar PSFs are used.

## A simulation at 0.5 keV (= 2.4 nm = 24 Angstrom)

First, we look at a simulation at one specific energy and see how the diffracted orders look on the detector and what we can learn from sub-aperturing. Later, we will repeat this analysis for a grid of photon energies, but it is useful to look in a little more detail for a single energy first to understand what is going on.

The convention that MARXS, our ray-trace code, uses for CAT gratings is to label the diffraction orders with negative numbers, so this is what we use in the following figures.

In [ ]:
orders = order_selector_Si.orders

fig = plt.figure()
ax = fig.add_subplot(111)
for k, p in phot.items():
    n_p = np.array([p[p['order'] == o]['probability'].sum() for o in orders])
    aeff = instrum[k].elements[0].area.to(u.cm**2) / 1e5 * n_p
    ax.plot(orders, aeff, label=labels[k])
    
ax.legend()
ax.set_xlim(-9,4)

In [ ]:
photons = phot['2050'][phot['2050']['probability'] > 0]
plt.scatter(photons['projcirc_y'], photons['projcirc_z'], c=photons['order'])
plt.colorbar(label='Diffraction order')
plt.xlabel('Dispersion direction in focal plane [mm]')
plt.ylabel('Cross-dispersion direction [mm]')

Figure above: Position of photons projected into the focal plane. Dispersion goes from left to right. Note that the scale of the x and y axis is *very* different!

We simuluate a Rowland spectrometer. As such, the CCD detectors are **not** in the focal plane, instead they follow the curved surface of the Rowland torus. This optimized for the spectral focus, i.e. it makes the orders narrow in dispersion direction, but at the cost of a wider distribution in the cross-dispersion direction. Only for the zeroth order (red, leftmost dot) do the imaging focus and the spectral focus agree and thus this order is circular as shown in the plot above. For CAT gratings, it is useful to use a *tilted* Rowland torus, which intersects the focal plane a second time, thus there is a second place (around 700 mm for the parameters chosen here) where the dispersed order is small in dispersion and cross-dispersion direction. This leads the fish-shaped distribution seen in the plot above. We now look at one of those orders in more detail.

In [ ]:
cmap = plt.get_cmap('hsv')

In [ ]:
facetpos = np.stack(instrum['5050'].elements[2].elem_pos)

fig = plt.figure(figsize=(15, 6))
ax1 = fig.add_subplot(121, aspect='equal')
ax1.scatter(facetpos[:, 1, 3], facetpos[:, 2, 3], c=np.rad2deg(facettab['5050']['facet_ang']),
            s=5, marker='s', cmap=cmap)
ax2 = fig.add_subplot(122)
photons = phot['2050']
ind = (photons['order'] == -5) #& (photons['proj_x'] > 0)
scat = ax2.scatter(photons['projcirc_y'][ind], photons['projcirc_z'][ind], 
            c=np.rad2deg(photons['facet_ang'])[ind], cmap=cmap)
#plt.xlim([-0.1, 0.1])
ax2.set_ylim([-1, 1])
plt.colorbar(scat, ax=ax2, label='Angle of grating facet')
ax2.set_xlabel('Dispersion direction in focal plane [mm]')
ax2.set_ylabel('Cross-dispersion direction [mm]')
ax1.set_xlabel('Distance from optical axis [mm]')
ax1.set_ylabel('Distance from optical axis [mm]')

In [ ]:
fig = plt.figure(figsize=(10, 8))

ax1 = fig.add_axes([.1, .3, .2, .4], aspect='equal')

w, h = .2, .4
x1 = .4
axes = [fig.add_axes([.4, .5, w, h])]
axes.append(fig.add_axes([.4 + w, .5, w, h]))
#axes.append(fig.add_axes([.4 - w, .5 - h, w, h]))
axes.append(fig.add_axes([.4 , .5 - h, w, h]))
axes.append(fig.add_axes([.4 + w, .5 - h, w, h]))

for i, k in enumerate(['2050', '5050', 'chirp', 'scat']):
    p = phot[k]
    ax = axes[i]
    ind = (p['order'] == -6) 
    scat = ax.scatter(p['projcirc_y'][ind], p['projcirc_z'][ind], 
            c=np.rad2deg(p['facet_ang'])[ind], cmap=cmap)
    #plt.colorbar(scat, ax=ax2, label='Angle of grating facet')
    ax.text(0.05, 0.92, labels[k], horizontalalignment='left',
            verticalalignment='center', transform=ax.transAxes) 
    ax.set_xlim([595.61, 596.29])
    ax.set_ylim([-.95, .95])
    ax.grid()

axes[3].set_xlabel('Dispersion [mm]')
axes[0].set_ylabel('Cross-dispersion [mm]') 
axes[2].set_ylabel('Cross-dispersion [mm]')
for i in [1, 3]:
    plt.setp(axes[i].get_yticklabels(), visible=False)

#ax1 = fig.add_subplot(231, aspect='equal')
ax1.scatter(facetpos[:, 1, 3], facetpos[:, 2, 3], c=np.rad2deg(facettab['5050']['facet_ang']),
            s=10, marker='s', cmap=cmap)
ax1.set_title('Gratings in aperture')
ax1.set_xlabel('along dispersion [mm]')
ax1.set_ylabel('along cross-dispersion [mm]')

fig.savefig(get_path('figures') + '/orders.png', dpi=300)
fig.savefig(get_path('figures') + '/orders.pdf', bbox_inches='tight')

The left plot shows the arrangement of the grating facets looking from end of the mirrors towards the focal plane. The dispersion direction is from left to right and cross-dispersion from top to bottom. There is no special meaning in that gap at the middle right. It's just where I started distributing the grating facets. The gap is just too small to place another set of facets so it's left empty. The distribution of facets can be improved to reduce the uncovered area in between, but that this stage it is not useful to spend to much detailed work on that. Note that the coloring scheme is different than in the 3D view on the interactive 3d website. Facets are colored according to the angle that the facet center has towards the positive dispersion direction.

The plot on the right shows a specific grating order on the detector (order 6). The detector is cylindrical following the Rowland circle. Intersection positions with this detector are the projected on the focal plane so that I can properly display them in a two-dimensional plot. Individual photons are colored according to the grating facet that they passed through. There are several things we can learn from this plot: First, note that the x and y axis are not to scale, this really is much longer in cross-dispersion direction (top to bottom) and narrow in the dispersion direction (left to right). Second, the absolute position along the dispersion direction is 75 cm from the focal point, so the dispersion angle is about 5 deg - significantly more than in Chandra or XMM-Newton. Thus, aberations that do not need to be considered for Chandra or XMM-Newton can be important here. 

So, we can now look at the distribution of colors and we see immediately that the distribution is not homogeneous. Photons that went through the cyan, green, and yellow colored gratings are distributed a narrower than the blue and red points. Thus, sub-aperturing *can* definitely increase the spectral resolving power. However, the pattern of the sub-aperturing is completely different than the "normal" pattern. Typically, sub-aperturing is done using only an area along the cross-dispersion direction (here the orange and light blue gratings). In contrast, here, we achieve the best spectral resolving power by using only gratings that are located along the forward dispersion direction.

This might be a surprising result at first glance, but thinking back to the assumptions that went into this simulation, it might seem a little more sensible. We set up the PSF such that there is no difference between in-plane and out-of-plane scatter and thus there is nothing special about the cross-dispersion direction. Instead the shape of the PSF is dominated by other things. Now, recall that this order is located about 75 cm from the optical axis; at the same time the facets go from -60 cm (red and blue) to +60 cm (light green). There is a considerable path length difference between those photons. The light green facets are located essentially "directly above" the order, while the red/blue ones are 1.5 m to the side; so it should not come a a complete surprise that the abbrations differ between those two groups.

In the following, I show how much better the spectral resolving power can be for different opening angles of the subaperture. That angle measures the angle between the center of a facet and the positive horizontal axis (the gap in the left image above). *There is no second mirrored sector*. For example, a subaperture angle of 30 degrees includes the cyan and light green gratings, 90 deg included light blue, cyan, green, yellow, and orange, and only the full aperture (suaperture angle 180 deg) includes all gratings, even the dark red and dark blue ones.

In [ ]:
fig = plt.figure(figsize=(4, 10))

xbins = np.arange(595.8, 596.2, .01)
angbins = np.arange(0, 181, 30.)
ind = (photlong['2050']['order'] == -6)
meanpos = np.mean(photlong['2050']['projcirc_y'][ind])

for i, k in enumerate(phot):
    p = photlong[k]
    ax = fig.add_subplot(len(phot) + 1, 1, i + 2)
    ax.set_prop_cycle(custom_cycler)
    ind = (p['order'] == -6) 
    hist2d = np.histogram2d(p['projcirc_y'][ind], np.abs(np.rad2deg(p['facet_ang'])[ind]), 
                        weights=p['probability'][ind], bins=[xbins, angbins])
    lines = ax.plot(0.5 * (xbins[:-1] + xbins[1:]) - meanpos, hist2d[0])

    #ax.text(0.05, 0.85, labels[k], horizontalalignment='left',
    #        verticalalignment='center', transform=ax.transAxes) 
    ax.set_ylabel(labels2line[k])
    ax.set_xlim([-0.1, .1])
    ax.set_ylim([0, None])
    ax.grid(axis='x')
    if i != (len(phot)-1):
        plt.setp(ax.get_xticklabels(), visible=False)
    else:
        ax.set_xlabel('Dispersion direction [mm]') 
    ax.tick_params(
    axis='y',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    left=False,      # ticks along the bottom edge are off
    right=False,         # ticks along the top edge are off
    labelleft=False) # labels along the bottom edge are off

ax = fig.add_subplot(6, 1, 1)
ax.legend(lines, [f'{angbins[i]:3.0f}-{angbins[i+1]:3.0f} deg' for i in range(angbins.shape[0]-1)], 
          ncol=2, title='Angle with respect to dispersion')
ax.axis('off')
    
fig.subplots_adjust(hspace=0)
fig.savefig(get_path('figures') + '/howtosubaperture.png', dpi=300)
fig.savefig(get_path('figures') + '/howtosuabperture.pdf', bbox_inches='tight')

This figure displays the same data in a different way: Photons are binned according to the position of the grating they passed through, and that distribution is shown along the dispersion axis, called the "line spread function" (LSF). The resolving power $R$ is given by the width of the photons' distribution; narrower peaks mean a better $R$. For all the scenarios, the photons passing through gratings located close to perpendicular to the dispersion direction (green line) have the sharpest peaks. The other lines differ from the green line in two ways: They are often wider and in some cases their peaks are shifted. The width of the distribution is due to the intrinsic mirror PSF (which is the same in these simulations over the entire aperture) and the fact that the gratings deviate from the Rowland torus. In our simulations, the gratings are placed such that the center of each grating matches the Rowland torus. Because of the tilt of the torus, gratings located in the range $60-90^\circ$ are almost tangential to the torus surface and have the smallest average deviation and thus the sharpest LSF. Larger gratings have larger average deviations than smaller gratings and thus the green LSF in the $50\;\mathrm{mm}\times 20\;\mathrm{mm}$ scenarios is narrower than in the $50\;\mathrm{mm}\times 50\;\mathrm{mm}$ scenario. Gratings located at larger or smaller angles cannot be tangential because they have to be rotated to match the blaze angle specification. Thus, they deviate more from the surface of the torus, leading to wider LSFs (lower $R$). Thus, these simulations show that a higher $R$ can be achieved if only certain parts of the aperture are filled with gratings. 

A second effect is the shift in the peak of the LSF between the green line and the other angles' ranges. This is due to our naive way of placing the gratings. Photons that pass the grating "outside" the Rowland torus travel further after they are diffracted and thus are detected at larger dispersion coordinates. Conversely, photons intersecting the grating "inside" the Rowland torus are detected at a smaller dispersion coordinate. At some locations, the gratings are mostly tangential to the Rowland torus, which means that almost the entire grating surface sits "outside" and thus photons are systematically send to larger dispersion coordinates; in other locations, half of the grating sits "inside". These shifts can be corrected by optimizing the grating position to reduce the average deviation from the Rowland torus and we will study this effect in more detail in future work. In our current simulations this is not done and thus the $R$ we derive is a lower bound to the value one would see with optimized grating placement. For example, for a simulation that covers the range in angles e.g.\ $30-120^\circ$ in the $50\;\mathrm{mm}\times 50\;\mathrm{mm}$ scenario, we would see the combined LSF from adding the orange, green, and red curve, which is wider than it would be for an optimized grating placement that effectively shifts all three curves to have the same peak before adding them up.

We now compare the different scenarios in the Figure to our baseline of $50\;\mathrm{mm}\times 20\;\mathrm{mm}$ gratings. Simply using larger gratings reduces $R$ as can be seen from comparing $50\;\mathrm{mm}\times 20\;\mathrm{mm}$ to the $50\;\mathrm{mm}\times 50\;\mathrm{mm}$. However, with a relatively simple chirp applied, even much larger gratings such as  $80\;\mathrm{mm}\times 160\;\mathrm{mm}$ can recover the best $R$ seen in the baseline case. With a chirp, essentially the same $R$ is observed for any grating position. If the entire aperture does not have to be filled with gratings, the grating locations can be chosen based on engineering constraints without compromising $R$.

The two scenarios $50\;\mathrm{mm}\times 20\;\mathrm{mm}$ and $50\;\mathrm{mm}\times 20\;\mathrm{mm}$ (scatter) use the same grating locations, but differ in the scatter properties of the mirror. If the mirror PSF is dominated by figure errors and scattering, then sub-aperturing will increase $R$ a lot more than in the conservative baseline $50\;\mathrm{mm}\times 20\;\mathrm{mm}$ scenario where the PSF is dominated by off-center errors.


## Resolving power and effective area

In [ ]:
def res_power_angle(photons, subaperangle, ang_0=0):
    resolvingpower = np.zeros((len(subaperangle), len(orders)))
    aeff_per_order = np.zeros_like(resolvingpower)
    for i, ang in enumerate(subaperangle):
        ind = np.abs(np.abs(photons['facet_ang']) - ang_0) < ang
        res, width, pos = resolvingpower_from_photonlist(photons[ind], orders, zeropos=0, col='projcirc_y')
        resolvingpower[i, :] = res
        aeff_per_order[i, :] = [photons['probability'][ind & (photons['order'] == o)].sum() for o in orders]
    aeff_per_order = aeff_per_order * instrum['2050'].elements[0].area.to(u.cm**2) / photons.meta['EXPOSURE'][0]
    return resolvingpower, aeff_per_order

In [ ]:
subaperangle = np.linspace(0, np.pi, 7)[1:]
tsubaperangle = np.linspace(0, np.pi/2, 7)[1:]

for k, p in phot.items():
    trespow, taeff = res_power_angle(p, tsubaperangle, np.pi/2)
    p.trespow = trespow
    p.taeff = taeff
    respow, aeff = res_power_angle(p, subaperangle)
    p.respow = respow
    p.aeff = aeff

In [ ]:
p = phot['2050']

fig = plt.figure() 

for i, ang in enumerate(tsubaperangle):
    plt.plot(orders, 
             p.trespow[i, :], 
             label='{:3.0f}'.format(np.rad2deg(ang)))
plt.legend(title='Subaperture\nangle [deg]', loc='upper left')
plt.ylabel('Resolving power')
plt.xlabel('Grating order')
plt.gca().invert_xaxis()

Dependence of the resolving power on the grating order

In [ ]:
ind0 = p['order'] == 0
for o in orders:
    ind = p['order'] == o
    if ind.sum() > 3:
        print(o, np.mean(p['detcirc_phi'][ind]) - np.mean(p['detcirc_phi'][ind0]))

In [ ]:
i =  (orders <= -4) & (orders >= -6)
fig = plt.figure(figsize=(4,3))
ax = fig.add_subplot(111)
ax.set_prop_cycle(custom_cycler)
for k, p in phot.items():
    ax.plot(np.average(p.trespow[:, i], weights=p.taeff[:, i], axis=1), 
             p.taeff[:, i].sum(axis=1), label=labels2line[k])

ax.legend()
ax.set_xlabel('Resolving power')
ax.set_ylabel('Effective area [cm$^2$]')
ax.set_xlim([None, 1.9e4])
fig.savefig(get_path('figures') + '/traderaeff.png', dpi=300, bbox_inches='tight')
fig.savefig(get_path('figures') + '/traderaeff.pdf', bbox_inches='tight')

Resolving power depending on the grating order (or diffraction angle). Sub-aperturing does little for low orders located close to the optical axis but can improve the resolving power up to a factor of three for higher orders. In practice though, only some orders receive a sufficient number of photons to do spectral analysis. Thus, we can take a weighted average of the resolving power for all orders where the resolving power for each order is weighted by the number of photons it receives.

## Simulations for different energies

Now, I run simulations with the same set-up as above for different input energies. The goal is not to produce a fine grid, but rather to set a few goalposts to see how the effect of sub-aperturing changes with energy.

In [ ]:
subaperangle = np.linspace(0, np.pi, 7)[1:]

In [ ]:
# energy = np.arange(0.3, 1.8, .02) * u.keV
# wavegrid = energy.to(u.nm, equivalencies=u.spectral())

In [ ]:
wavegrid = np.arange(0.7, 4., 0.05) * u.nm
energy = wavegrid.to(u.keV, equivalencies=u.spectral())

In [ ]:
phot_en = []

for i, e in enumerate(energy):
    n = 1e4 * u.s
    if e.value > 1.25:
        n=5e4 * u.s
    
    p = run_monoenergetic_simulation(instrum['2050'], e, n)
    p = join(p, facettab['2050'])
    phot_en.append(p)


In [ ]:
for p in phot_en:
    ind = p['CCD_ID'] >= 0
    trespow, taeff = res_power_angle(p[ind], tsubaperangle, np.pi/2)
    p.trespow = trespow
    p.taeff = taeff
    respow, aeff = res_power_angle(p[ind], subaperangle)
    p.respow = respow
    p.aeff = aeff

In [ ]:
from matplotlib.ticker import MaxNLocator

fig = plt.figure(figsize=(15, 15))

indorders = np.arange(4, 16)
lenord = indorders.shape[0]

selected_energies = [1, 20, 50]
lenen = len(selected_energies)

axes = np.empty((lenen, lenord), dtype=object)


for i, e in enumerate(selected_energies):
    axes[i, 0] = fig.add_subplot(lenen, lenord, lenord * i + 1)
    axes[i, 0].set_ylabel(str(energy[i]))
    plt.setp(axes[i, 0].get_yticklabels(), visible=False)
    for j in range(1, len(indorders)):
        axes[i, j] = fig.add_subplot(lenen, lenord, lenord * i + j + 1) #, sharey=axes[i, 0])
        plt.setp(axes[i, j].get_yticklabels(), visible=False)
        
for i, e in enumerate(selected_energies):
    p = phot_en[i]
    for j, o in enumerate(indorders):      
        order = orders[o]
        ind = (p['order'] == order) #& (np.abs(phot['proj_y']) < 10)
        axes[i, j].scatter(p['proj_x'][ind], p['proj_y'][ind], 
                           c=np.rad2deg(p['facet_ang'])[ind],
                           edgecolors='none', cmap=cmap)
        

for j, o in enumerate(indorders):
    axes[0, j].set_title('order {}'.format(orders[o]))
    
#axes[-1, 5].set_xlabel('position on dispersion direction [mm]. Note that the scale differs for every plot.')
    
fig.subplots_adjust(wspace=0)

The plot above shows the PSF for different grating orders and energies. The energy for each row is listed on the left, and then the PSF is shown for order 0, -1, -2 ,... from left to right. THe x-axis in each panel has a different scaling and shows the position in dispersion direction (in mm) measured from the optical axis. Dots are colored according to the angle of the facet that the photon went through using the same color scale as above. For each order, the y axis is scaled differently. As shown in the fish-shaped plot above, the width in the y direction ranges from 0.02 mm for order 0 to a a few mm for higher orders. Using a the same scaling in y direction for all orders in a row would hide several of the details for the PSFs I'm going to discuss now, but keep in mind that the scaling of x and y axis is not the same. For exmaple, all zeroth orders are really circular and not elliptical.

Let me know highlight a few insteresting aspects about the use of subaperturing for different energies and orders. This discussion here is qualitatively and I try to explain why things look the way they do and later I will show the actual number for the resolving power. Looking at the zeroth order for any energy, all the colors are well mixed in the plot and sub-aperturing will not do anything to change that. This is simply the result of our setup, where the scattering of the mirror is the same in every direction. At 0.3 keV most of the photons are found in order -3 and -4. In both cases, we see that sub-aperturing as discussed in detail above (selecting the angles for cyan, green, and yellow) would significantly narrow the order and thus improve the spectral resolving power. It is interesting to note that the color scheme seems flipped between order -3 and -4. In order -3 the blue-ish photons are found on the top and the reddish photons at the bottom, in order -4 that is reversed. The Rowland torus optimizes the focussing in the dispersion direction, but the focus in the cross-dispersion direction is different. In one case the detector is located below the focus in cross-dispersion direction (so in y direction photons have passed through the focus and spread out again), in the other case it is above the cross-dispersion focus. This flip happens around 62 cm, where the 7 order of 0.6 keV photons is located. Photons of higher energy (and thus lower wavelength) are not dispersed this far out and thus there is no flip in the plots for 1.0, 1.4, and 1.8 keV.

In general, for higher energies where the grating orders are located closer to the optical axis, the distribution of the photons in each order depends less on the angle of the facet that they went through and thus sub-aperturing cannot do much to improve the spectral resolving power. The simulations for 1.4 and 1.8 keV photons show this. At these energies, the efficiency also peaks at much lower orders and the relevant signal is only a few cm from the optical axis (for example, order -2 for 1.8 keV is at 59 mm = 5.9 cm. 

At 1 keV, a significant number of photons are seen in very low orders (-1 to -3) where sub-aperturing is not relevant, but then again there are many photons in order -9. If Lynx uses sub-aperturing, then one could analyze the photons in order 9 for high-resolution work, if instead a larger number of photons is required for the science goal, but spectral resolving power is not crucial, then the observer could include the lower orders in the analysis. However, for simplicity, I will just average the resolving power for all orders when I show plots of resolving power vs sub-aperturing angle below.

I want to point out one more feature: While the signal of the cyan, green, and yellow photons is generally is approximately centered on the same position as the wider distribution for the red and blue photons, this is not the case around 40-50 cm from the optical axis (order -2 for 0.3 keV; order -5 for 0.6 keV; order -9 for 1 keV).


In [ ]:
resolvingpower_en = np.zeros((len(subaperangle), len(energy)))

for i, e in enumerate(energy):
    p = phot_en[i]
    zeropos = np.mean(p['detcirc_phi'][p['order']==0])
    resolvingpower = np.zeros((len(subaperangle), len(orders)))
    for j, ang in enumerate(subaperangle):
        ind = p['CCD_ID'] >=0
        res, width, pos = resolvingpower_from_photonlist(p[np.abs(p['facet_ang']) < ang],
                                                         orders, col='detcirc_phi', zeropos=zeropos)
        resolvingpower[j, :] = res
        
    resolvingpower = np.ma.masked_invalid(resolvingpower)
    # Mask out the zeros order which always has resolving power 0 
    resolvingpower[:, 0] = np.ma.masked
    res = np.ma.average(resolvingpower, axis=1, 
                        weights=order_selector_Si.probabilities([e], [0], [lynx.conf['blazeang']])[1].flatten())
    resolvingpower_en[:, i] = res
        

In [ ]:
for i, ang in enumerate(subaperangle):
    plt.plot(energy, resolvingpower_en[i, :], label='{:3.0f}'.format(np.rad2deg(ang)))
plt.legend(title='Subaperture\nangle [deg]')
plt.ylabel('Resolving power')
plt.xlabel('Photon energy [keV]')

This plot shows the spectral resolving power vs energy for different sub-aperturing angles. When calculating the average, different orders are weighted according to the number of photons they receive. Sub-aperturing will not increase the resolving power for high energies, but sacrificing 2/3 of the effective area in a 60 degree sub-aperture angle would increase the resolving power at low energies by almost a factor of three.

In [ ]:
en_trespos = np.stack([p.trespow for p in phot_en])
en_taeff = np.stack([p.taeff for p in phot_en])
en_trespos.shape

In [ ]:
fig = plt.figure(figsize=(4,3))
ax = fig.add_subplot(111)

for i, ang in enumerate(tsubaperangle):
    ax.plot(wavegrid, en_taeff[:, i, :].sum(axis=1), label='{:3.0f}'.format(np.rad2deg(ang)))
#ax.legend(title='Subaperture\nangle [deg]')
ax.set_ylabel('Effective Area [cm$^2$]')
ax.set_xlabel('wavelength [nm]')

fig.savefig(get_path('figures') + '/aeff_en.png', 
            dpi=300, bbox_inches='tight')
fig.savefig(get_path('figures') + '/aeff_en.pdf', bbox_inches='tight')

In [ ]:
# There are Nans in there which srew up the average
en_trespos[np.isnan(en_trespos)] = 0

In [ ]:
fig = plt.figure(figsize=(4,3))
ax = fig.add_subplot(111)

for i, ang in enumerate(tsubaperangle):
    ax.plot(wavegrid, np.average(en_trespos[:, i, :], axis=1, weights=en_taeff[:, i,:]), 
             label='{:3.0f} %'.format(np.rad2deg(ang)*4/360*100))
ax.legend(title='Aperture area covered by gratings', ncol=2)
ax.set_ylabel('Resolving power')
ax.set_xlabel('wavelength [nm]')
ax.set_ylim([0, 1.2e4])
ax.set_xlim([.5, 3.5])

fig.savefig(get_path('figures') + '/rew_en.png', dpi=300, bbox_inches='tight')
fig.savefig(get_path('figures') + '/rew_en.pdf', bbox_inches='tight')

In [ ]:
# Combine both plots into a single file as required for JATIS
fig = plt.figure(figsize=(8, 3))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

for i, ang in enumerate(tsubaperangle):
    ax1.plot(wavegrid, en_taeff[:, i, :].sum(axis=1), label='{:3.0f}'.format(np.rad2deg(ang)))
#ax.legend(title='Subaperture\nangle [deg]')
ax1.set_ylabel('Effective Area [cm$^2$]')
ax1.set_xlabel('wavelength [nm]')
ax1.set_xlim([.5, 3.5])

for i, ang in enumerate(tsubaperangle):
    ax2.plot(wavegrid, np.average(en_trespos[:, i, :], axis=1, weights=en_taeff[:, i,:]), 
             label='{:3.0f} %'.format(np.rad2deg(ang)*4/360*100))
ax2.legend(title='Aperture area covered by gratings', ncol=2)
ax2.set_ylabel('Resolving power')
ax2.set_xlabel('wavelength [nm]')
ax2.set_ylim([0, 1.2e4])
ax2.set_xlim([.5, 3.5])

fig.subplots_adjust(wspace=.3)
fig.savefig(get_path('figures') + '/fig12.png', 
            dpi=300, bbox_inches='tight')
fig.savefig(get_path('figures') + '/fig12.pdf', bbox_inches='tight')

In [ ]:
np.rad2deg(tsubaperangle)*4/360*100

In [ ]:
tsave = Table()
tsave['wave'] = wavegrid
tsave['R'] = np.average(en_trespos[:, 3, :], axis=1, weights=en_taeff[:, i,:])
tsave['Aeff'] = en_taeff[:, 3, :].sum(axis=1)
tsave.write(get_path('figures') + '/lynxRaeff.ecsv', format='ascii.ecsv')

## Future work

Much remains to be done to study this in more detail. There are several parameters in the simulation that I fixed to certain values based on experience with simulating other observatories, but that remain to be studied in the context of Lynx. An incomplete list is:

- **Torus tilt** The torus is tilted by a little more than twice the facet blaze angle. There are two free parameters in here which Heilmann et al. 2010 call "hinge points". I don't think that the exact numbers are critical for the answer, but that should be checked.